In [1]:
import os
import json

from pymongo import MongoClient

Set environment variables and setup simulated vasp run

In [2]:
#set the path to vasp pseudopotential files
os.environ["VASP_PSP_DIR"] = os.path.join(os.getcwd(), "Si_bandstructure_runs")

# Structure and input set

In [3]:
from pymatgen.util.testing import PymatgenTest
from pymatgen.io.vasp.sets import MPRelaxSet

struct_si = PymatgenTest.get_structure("Si")
vis = MPRelaxSet(struct_si, force_gamma=True)

# Define Fireworks and Workflow

The workflow consists of  3 fireworks:
- structure optimization
- static calculation
- refined band structure calculation

In [4]:
from fireworks import Workflow
from matmethods.vasp.fireworks.core import OptimizeFW, StaticFW, NonSCFFW

fw1 = OptimizeFW(struct_si, vasp_input_set=vis, vasp_cmd=None, db_file=">>db_file<<")
fw2 = StaticFW(struct_si, vasp_input_set=vis, vasp_cmd=None, db_file=">>db_file<<", parents=[fw1])
fw3 = NonSCFFW(struct_si, vasp_cmd=None, db_file=">>db_file<<", parents=[fw2], mode="uniform")
#fw4 = NonSCFFW(struct_si, vasp_cmd=None, db_file=">>db_file<<", parents=[fw2], mode="line")

fws = [fw1, fw2, fw3]

wf = Workflow(fws)

# Emulate VASP

In [5]:
from matmethods.vasp.vasp_powerups import use_fake_vasp

reference_dir = os.path.join(os.getcwd(), "Si_bandstructure_runs")
#reference_dir = os.path.abspath(os.path.join("..","..","matproj/MatMethods/matmethods/vasp/tests", "reference_files"))

si_ref_dirs = {"structure optimization": os.path.join(reference_dir, "Si_structure_optimization"),
             "static": os.path.join(reference_dir, "Si_static"),
             "nscf uniform": os.path.join(reference_dir, "Si_nscf_uniform"),
             "nscf line": os.path.join(reference_dir, "Si_nscf_line")}

wf_fake = use_fake_vasp(wf, si_ref_dirs)

In [6]:
# total number of fireworks in the workflow
print len(wf_fake.as_dict()["fws"])

3


# Connect to Launchpad and reset

In [7]:
import datetime
from fireworks import LaunchPad

lp = LaunchPad.from_file("my_launchpad.yaml")
lp.reset(datetime.datetime.now().strftime('%Y-%m-%d'))

2016-08-04 12:19:14,806 INFO Performing db tune-up
2016-08-04 12:19:14,818 INFO LaunchPad was RESET.


# Add Workflow

In [8]:
lp.add_wf(wf_fake)

2016-08-04 12:19:16,193 INFO Added a workflow. id_map: {-1: 3, -3: 1, -2: 2}


{-3: 1, -2: 2, -1: 3}

In [9]:
def get_status():
    for id in lp.get_fw_ids():
        fw = lp.get_fw_by_id(id)
        print "Name:{}, State:{}".format(fw.name, fw.state)

def set_launch_dir(launch_dir):
    os.mkdir(launch_dir)
    os.chdir(launch_dir)

print firework names and their states

In [10]:
get_status()

Name:Si-nscf uniform, State:WAITING
Name:Si-static, State:WAITING
Name:Si-structure optimization, State:READY


# Launch Structure optimization firework

In [11]:
from fireworks import FWorker
from fireworks.core.rocket_launcher import rapidfire, launch_rocket

db_file = os.path.join(os.getcwd(), "db.json")
#rapidfire(lp, fworker=FWorker(env={"db_file": db_file}))

In [12]:
set_launch_dir("structure_optimization")
launch_rocket(lp, fworker=FWorker(env={"db_file": db_file}))

2016-08-04 12:19:22,939 INFO Launching Rocket
2016-08-04 12:19:23,177 INFO RUNNING fw_id: 3 in directory: /home/ray/Software/workshop-2016/matmethods/structure_optimization
2016-08-04 12:19:23,180 INFO Task started: {{matmethods.vasp.firetasks.write_inputs.WriteVaspFromIOSet}}.
2016-08-04 12:19:23,197 INFO Task completed: {{matmethods.vasp.firetasks.write_inputs.WriteVaspFromIOSet}} 
2016-08-04 12:19:23,198 INFO Task started: {{matmethods.vasp.firetasks.run_calc.RunVaspFake}}.
2016-08-04 12:19:23,234 INFO matmethods.vasp.firetasks.run_calc RunVaspFake: verified inputs successfully
2016-08-04 12:19:23,243 INFO matmethods.vasp.firetasks.run_calc RunVaspFake: ran fake VASP, generated outputs
2016-08-04 12:19:23,244 INFO Task completed: {{matmethods.vasp.firetasks.run_calc.RunVaspFake}} 
2016-08-04 12:19:23,246 INFO Task started: {{matmethods.common.firetasks.glue_tasks.PassCalcLocs}}.
2016-08-04 12:19:23,251 INFO Task completed: {{matmethods.common.firetasks.glue_tasks.PassCalcLocs}} 
201

True

In [13]:
get_status()

Name:Si-nscf uniform, State:WAITING
Name:Si-static, State:READY
Name:Si-structure optimization, State:COMPLETED


# Launch static vasp calculation firework

In [14]:
set_launch_dir("../static")

launch_rocket(lp, fworker=FWorker(env={"db_file": db_file}))

2016-08-04 12:19:30,116 INFO Launching Rocket
2016-08-04 12:19:30,479 INFO RUNNING fw_id: 2 in directory: /home/ray/Software/workshop-2016/matmethods/static
2016-08-04 12:19:30,481 INFO Task started: {{matmethods.vasp.firetasks.glue_tasks.CopyVaspOutputs}}.
2016-08-04 12:19:30,532 INFO Task completed: {{matmethods.vasp.firetasks.glue_tasks.CopyVaspOutputs}} 
2016-08-04 12:19:30,532 INFO Task started: {{matmethods.vasp.firetasks.write_inputs.WriteVaspStaticFromPrev}}.
2016-08-04 12:19:30,589 INFO Task completed: {{matmethods.vasp.firetasks.write_inputs.WriteVaspStaticFromPrev}} 
2016-08-04 12:19:30,590 INFO Task started: {{matmethods.vasp.firetasks.run_calc.RunVaspFake}}.
2016-08-04 12:19:30,617 INFO matmethods.vasp.firetasks.run_calc RunVaspFake: verified inputs successfully
2016-08-04 12:19:30,635 INFO matmethods.vasp.firetasks.run_calc RunVaspFake: ran fake VASP, generated outputs
2016-08-04 12:19:30,636 INFO Task completed: {{matmethods.vasp.firetasks.run_calc.RunVaspFake}} 
2016-08

True

In [15]:
get_status()

Name:Si-nscf uniform, State:READY
Name:Si-static, State:COMPLETED
Name:Si-structure optimization, State:COMPLETED


# Launch vasp bandstructure calculation firework

In [16]:
set_launch_dir("../bandstructure")

launch_rocket(lp, fworker=FWorker(env={"db_file": db_file}))

2016-08-04 12:19:36,011 INFO Launching Rocket
2016-08-04 12:19:36,260 INFO RUNNING fw_id: 1 in directory: /home/ray/Software/workshop-2016/matmethods/bandstructure
2016-08-04 12:19:36,264 INFO Task started: {{matmethods.vasp.firetasks.glue_tasks.CopyVaspOutputs}}.
2016-08-04 12:19:36,718 INFO Task completed: {{matmethods.vasp.firetasks.glue_tasks.CopyVaspOutputs}} 
2016-08-04 12:19:36,719 INFO Task started: {{matmethods.vasp.firetasks.write_inputs.WriteVaspNSCFFromPrev}}.
2016-08-04 12:19:36,857 INFO Task completed: {{matmethods.vasp.firetasks.write_inputs.WriteVaspNSCFFromPrev}} 
2016-08-04 12:19:36,858 INFO Task started: {{matmethods.vasp.firetasks.run_calc.RunVaspFake}}.
2016-08-04 12:19:36,890 INFO matmethods.vasp.firetasks.run_calc RunVaspFake: verified inputs successfully
2016-08-04 12:19:36,900 INFO matmethods.vasp.firetasks.run_calc RunVaspFake: ran fake VASP, generated outputs
2016-08-04 12:19:36,901 INFO Task completed: {{matmethods.vasp.firetasks.run_calc.RunVaspFake}} 
2016

True

# Retrieve and Verify

In [17]:
def get_task_collection(db_file):
    """
    connect to the database and return task collection
    """
    with open(db_file) as f:
        creds = json.loads(f.read())
        conn = MongoClient(creds["host"], creds["port"])
        db = conn[creds["database"]]
        if "admin_user" in creds:
            db.authenticate(creds["admin_user"], creds["admin_password"])
        return db[creds["collection"]]

In [18]:
from pymongo import DESCENDING

d1 = get_task_collection(db_file).find_one({"task_label": "structure optimization"}, sort=[("_id", DESCENDING)])

d2 = get_task_collection(db_file).find_one({"task_label": "static"}, sort=[("_id", DESCENDING)])

d3 = get_task_collection(db_file).find_one({"task_label": "nscf uniform"}, sort=[("_id", DESCENDING)])

d4 = get_task_collection(db_file).find_one({"task_label": "nscf line"}, sort=[("_id", DESCENDING)])

In [19]:
d1["output"]["bandgap"] # 0.65

0.6500000000000004